# Visualization Module Demo

This notebook demonstrates all visualization capabilities for the M2M Sleep-Based Simulator.

**Task 2.3: Visualization Integration**

Features:
- Lifetime vs. delay scatter plots
- Lifetime vs. parameter curves
- Delay vs. parameter curves
- Queue evolution over time
- Energy breakdown pie charts
- State occupation pie charts
- Energy depletion curves
- Trade-off comparison plots
- Interactive parameter exploration with ipywidgets

**Date:** February 10, 2026

In [ ]:
# Setup path
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from src import (
    Simulator, SimulationConfig,
    PowerModel, PowerProfile,
    ParameterSweep, ScenarioExperiments,
    SimulationVisualizer, InteractiveVisualizer, PlotConfig,
    plot_parameter_sweep_summary, save_figure
)

## 1. Basic Setup

Create a base configuration for our demonstrations.

In [ ]:
# Get power profile
power_rates = PowerModel.get_profile(PowerProfile.NR_MMTC)

# Create base configuration
base_config = SimulationConfig(
    n_nodes=50,
    arrival_rate=0.02,
    transmission_prob=0.1,
    idle_timer=10,
    wakeup_time=2,
    initial_energy=5000,
    power_rates=power_rates,
    max_slots=50000,
    seed=42
)

print("Base configuration:")
print(f"  Nodes: {base_config.n_nodes}")
print(f"  Arrival rate (λ): {base_config.arrival_rate}")
print(f"  Transmission prob (q): {base_config.transmission_prob}")
print(f"  Idle timer (ts): {base_config.idle_timer}")
print(f"  Wake-up time (tw): {base_config.wakeup_time}")

## 2. Run a Single Simulation with Time Series

First, let's run a single simulation with time series tracking enabled to demonstrate queue and energy evolution plots.

In [ ]:
# Run simulation with time series tracking
sim = Simulator(base_config)
result = sim.run_simulation(track_history=True, verbose=True)

print("\nSimulation Results:")
print(f"  Mean lifetime: {result.mean_lifetime_years:.2f} years")
print(f"  Mean delay: {result.mean_delay:.2f} slots")
print(f"  Throughput: {result.throughput:.4f}")
print(f"  Success probability: {result.empirical_success_prob:.4f}")

## 3. Comprehensive Summary Dashboard

Create a multi-panel dashboard showing various aspects of the simulation.

In [ ]:
# Create visualizer
viz = SimulationVisualizer()

# Create comprehensive dashboard
fig = viz.create_summary_dashboard(result, include_time_series=True, figsize=(18, 12))
plt.show()

## 4. Individual Plot Examples

### 4.1 Energy Breakdown Pie Chart

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
viz.plot_energy_breakdown_pie(result, ax=ax)
plt.show()

### 4.2 State Occupation Pie Chart

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
viz.plot_state_occupation_pie(result, ax=ax)
plt.show()

### 4.3 Queue Evolution Over Time

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
viz.plot_queue_evolution(result, sample_rate=100, ax=ax)
plt.show()

### 4.4 Energy Depletion Over Time

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
viz.plot_energy_depletion(result, sample_rate=100, ax=ax)
plt.show()

## 5. Parameter Sweep Visualizations

### 5.1 Sweep Transmission Probability (q)

This is a key trade-off: higher q reduces delay but increases energy consumption.

In [ ]:
# Run q sweep with fewer replications for demo speed
print("Running transmission probability sweep...")
q_results = ParameterSweep.sweep_transmission_prob(
    base_config,
    q_values=[0.02, 0.05, 0.1, 0.15, 0.2],
    n_replications=10,
    verbose=True
)

In [ ]:
# Create comprehensive summary for q sweep
fig = plot_parameter_sweep_summary(q_results, param_name="q", figsize=(18, 12))
plt.show()

### 5.2 Individual Q Sweep Plots

In [ ]:
# Lifetime vs. delay scatter - KEY TRADE-OFF PLOT
fig, ax = plt.subplots(figsize=(10, 6))
viz.plot_lifetime_vs_delay_scatter(
    q_results,
    param_name="q",
    ax=ax
)
plt.show()

In [ ]:
# Lifetime vs. q
fig, ax = plt.subplots(figsize=(10, 6))
viz.plot_lifetime_vs_parameter(
    q_results,
    param_name="q",
    xlabel="Transmission Probability (q)",
    ax=ax
)
plt.show()

In [ ]:
# Mean delay vs. q
fig, ax = plt.subplots(figsize=(10, 6))
viz.plot_delay_vs_parameter(
    q_results,
    param_name="q",
    delay_type="mean",
    xlabel="Transmission Probability (q)",
    ax=ax
)
plt.show()

In [ ]:
# Tail delay (95th percentile) vs. q
fig, ax = plt.subplots(figsize=(10, 6))
viz.plot_delay_vs_parameter(
    q_results,
    param_name="q",
    delay_type="tail_95",
    xlabel="Transmission Probability (q)",
    ax=ax
)
plt.show()

### 5.3 Sweep Idle Timer (ts)

The idle timer controls when nodes go to sleep - a critical parameter for the lifetime-latency trade-off.

In [ ]:
# Run ts sweep
print("Running idle timer sweep...")
ts_results = ParameterSweep.sweep_idle_timer(
    base_config,
    ts_values=[1, 5, 10, 20, 50],
    n_replications=10,
    verbose=True
)

In [ ]:
# Create comprehensive summary for ts sweep
fig = plot_parameter_sweep_summary(ts_results, param_name="ts", figsize=(18, 12))
plt.show()

In [ ]:
# Lifetime vs. delay scatter for different ts values
fig, ax = plt.subplots(figsize=(10, 6))
viz.plot_lifetime_vs_delay_scatter(
    ts_results,
    param_name="ts (idle timer)",
    ax=ax
)
plt.show()

## 6. Scenario Comparison: Low-Latency vs. Battery-Life Prioritization

This demonstrates the fundamental trade-off in M2M systems.

In [ ]:
# Run scenario experiments
print("Running scenario comparison...")
scenario_results = ScenarioExperiments.compare_latency_vs_battery(
    base_config,
    n_replications=10,
    verbose=True
)

In [ ]:
# Plot trade-off comparison
fig, ax = plt.subplots(figsize=(10, 6))
viz.plot_tradeoff_comparison(
    scenario_results,
    metric_x="mean_delay",
    metric_y="mean_lifetime_years",
    xlabel="Mean Delay (slots)",
    ylabel="Mean Lifetime (years)",
    title="Low-Latency vs. Battery-Life Prioritization",
    ax=ax
)
plt.show()

In [ ]:
# Print detailed comparison
for scenario_name, results in scenario_results.items():
    mean_lifetime = np.mean([r.mean_lifetime_years for r in results])
    mean_delay = np.mean([r.mean_delay for r in results])
    mean_throughput = np.mean([r.throughput for r in results])
    
    print(f"\n{scenario_name}:")
    print(f"  Mean lifetime: {mean_lifetime:.2f} years")
    print(f"  Mean delay: {mean_delay:.2f} slots")
    print(f"  Throughput: {mean_throughput:.4f}")

## 7. Custom Plot Styling

Demonstrate custom plot configurations.

In [ ]:
# Create custom plot config
custom_config = PlotConfig(
    figsize=(12, 7),
    dpi=120,
    title_fontsize=16,
    label_fontsize=14,
    colormap='plasma'
)

# Create visualizer with custom config
custom_viz = SimulationVisualizer(custom_config)

# Create plot with custom styling
fig, ax = plt.subplots(figsize=custom_config.figsize, dpi=custom_config.dpi)
custom_viz.plot_lifetime_vs_delay_scatter(
    q_results,
    param_name="q",
    title="Custom Styled Trade-off Plot",
    ax=ax
)
plt.show()

## 8. Save Figures

Demonstrate saving publication-quality figures.

In [ ]:
# Create a plot
fig, ax = plt.subplots(figsize=(10, 6))
viz.plot_lifetime_vs_delay_scatter(
    q_results,
    param_name="q",
    ax=ax
)

# Save in multiple formats
save_figure(fig, "lifetime_vs_delay_tradeoff", formats=['png', 'pdf'], dpi=300)
plt.show()

## 9. Interactive Parameter Exploration

Use ipywidgets for real-time parameter exploration (requires `ipywidgets`).

**Note:** This feature requires a reduced max_slots for responsive interaction.

In [ ]:
# Create config for interactive exploration (shorter run for responsiveness)
interactive_config = SimulationConfig(
    n_nodes=50,
    arrival_rate=0.02,
    transmission_prob=0.1,
    idle_timer=10,
    wakeup_time=2,
    initial_energy=2000,  # Lower for faster runs
    power_rates=power_rates,
    max_slots=10000,  # Reduced for interactive responsiveness
    seed=42
)

# Create interactive visualizer
interactive_viz = InteractiveVisualizer(interactive_config)

# Create interactive explorer (uncomment to use)
# Note: This requires ipywidgets installed: pip install ipywidgets
# interactive_viz.create_interactive_explorer(
#     q_range=(0.01, 0.3),
#     ts_range=(1, 50),
#     n_range=(10, 100),
#     metrics_to_plot=['mean_lifetime_years', 'mean_delay', 'throughput']
# )

## 10. Multi-Panel Comparison

Create a publication-ready multi-panel figure comparing different aspects.

In [ ]:
# Create comprehensive comparison figure
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Panel 1: Lifetime vs delay for q sweep
viz.plot_lifetime_vs_delay_scatter(q_results, "q", ax=axes[0, 0])

# Panel 2: Lifetime vs q
viz.plot_lifetime_vs_parameter(q_results, "q", ax=axes[0, 1])

# Panel 3: Delay vs q
viz.plot_delay_vs_parameter(q_results, "q", ax=axes[0, 2])

# Panel 4: Lifetime vs delay for ts sweep
viz.plot_lifetime_vs_delay_scatter(ts_results, "ts", ax=axes[1, 0])

# Panel 5: Lifetime vs ts
viz.plot_lifetime_vs_parameter(ts_results, "ts", ax=axes[1, 1])

# Panel 6: Delay vs ts
viz.plot_delay_vs_parameter(ts_results, "ts", ax=axes[1, 2])

plt.tight_layout()
plt.show()

## Summary

This notebook demonstrated:

1. ✅ **Energy breakdown pie charts** - showing where energy is consumed
2. ✅ **State occupation pie charts** - showing time in each state
3. ✅ **Queue evolution plots** - showing queue dynamics over time
4. ✅ **Energy depletion plots** - showing battery drainage
5. ✅ **Lifetime vs. delay scatter plots** - KEY TRADE-OFF visualization
6. ✅ **Parameter sweep summaries** - comprehensive multi-panel analysis
7. ✅ **Scenario comparisons** - low-latency vs. battery-life prioritization
8. ✅ **Custom styling** - publication-quality plot configuration
9. ✅ **Figure saving** - export in multiple formats (PNG, PDF)
10. ✅ **Interactive exploration** - real-time parameter adjustment (with ipywidgets)

All plots can be customized with:
- Custom colors, sizes, fonts
- Different metrics and parameters
- Confidence intervals
- Multiple scenarios

**Task 2.3 Complete!**